In [ ]:
using CounterfactualExplanations
using CounterfactualExplanations.Data: load_mnist
using CounterfactualExplanations.Models: load_mnist_mlp
using Images
using MLDatasets
using MLDatasets: convert2image
using Plots

## Motivation

Counterfactual Explanations are a powerful, flexible and intuitive way to not only explain black-box models, but also enable affected individuals to challenge them though the means of Algorithmic Recourse. 

### From Adversarial Examples to Counterfactual Explanations

Most state-of-the-art approaches to generating Counterfactual Explanations (CE) rely on gradient descent in the feature space. The key idea is to perturb inputs $x\in\mathcal{X}$ into a black-box model $f: \mathcal{X} \mapsto \mathcal{Y}$ in order to change the model output $f(x)$ to some pre-specified target value $t\in\mathcal{Y}$. Formally, this boils down to defining some loss function $\ell(f(x),t)$ and taking gradient steps in the minimizing direction. The so generated counterfactuals are considered valid as soon as the predicted label matches the target label. A stripped down counterfactual explanation is therefore little different from an adversarial example.

> You may not like it, but this is what counterfactuals look like


In [ ]:
# Data:
counterfactual_data = load_mnist()
X, y = CounterfactualExplanations.DataPreprocessing.unpack_data(counterfactual_data)
input_dim, n_obs = size(counterfactual_data.X)
M = load_mnist_mlp()

# Target:
factual_label = 8
x = reshape(X[:,rand(findall(predict_label(M, counterfactual_data).==factual_label))],input_dim,1)
target = 3
factual = predict_label(M, counterfactual_data, x)[1]

# Search:
n_ce = 3
generator = GenericGenerator()
ces = generate_counterfactual(x, target, counterfactual_data, M, generator; num_counterfactuals=n_ce)

In [ ]:
image_size = 300
p1 = plot(
    convert2image(MNIST, reshape(x,28,28)),
    axis=nothing, 
    size=(image_size, image_size),
    title="Factual"
)
plts = [p1]

for i in eachindex(ces)
    ce = ces[i]
    plt = plot(
        convert2image(MNIST, reshape(CounterfactualExplanations.counterfactual(ce),28,28)),
        axis=nothing, 
        size=(image_size, image_size),
        title="Counterfactual $i"
    )
    plts = [plts..., plt]
end

plt = plot(plts...; size=(image_size * (n_ce + 1),image_size), layout=(1,(n_ce + 1)))

The crucial difference between adversarial examples and counterfactuals is one of intent. While adversarial examples are typically intened to go unnoticed, counterfactuals in the context of Explainable AI are generally sought to be "plausible" or "realistic". To fulfill this latter goal, researchers have come up with a myriad of ways. @joshi2019realistic were among the first to suggest that instead of searching counterfactuals in the feature space, we can instead traverse a latent embedding learned by a surrogate generative model. This ensures that the generated counterfactuals comply with the (learned) data-generating process (DGB). Similarly, @poyiadzi2020face use density ...

- Show DiCE for weak MLP
- Show Latent for same weak MLP
- Latent can be manipulated: 
    - train biased model
    - train VAE with biased variable removed/attacked (use Boston housing dataset)
    - hypothesis: will generate bias-free explanations

::: {#prp-surrogate}

## Avoid Surrogates

Since we are in the business of explaining a black-box model, the task of learning realistic representations of the data should not be reallocated from the model itself to some surrogate model.

:::

## Introduction to Conformal Prediction

- distribution-free, model-agnostic and scalable approach to predictive uncertainty quantification

### Post-hoc

- Take any fitted model and turn it into a conformal model using calibration data.

### Intrinsic --- Conformal Training [MAYBE]

- Model explicitly trained for conformal prediction.

## Conformal Counterfactuals

- Realistic counterfactuals by minimizing predictive uncertainty [@schut2021generating].
- Problem: restricted to Bayesian models.
- Solution: post-hoc predictive uncertainty quantification. 
- Conformal prediction is instance-based. So is CE. 
- Does the coverage guarantee carry over to counterfactuals?

### Research Questions

- Is CP alone enough to ensure realistic counterfactuals?
- Do counterfactuals improve further as the models get better?
- Do counterfactuals get more realistic as coverage
- What happens as we vary coverage and setsize?
- What happens as we improve the model robustness?
- What happens as we improve the model's ability to incorporate predictive uncertainty (deep ensemble, laplace)?

## Experiments

- Maybe: conformalised Laplace
- Benchmarking:
    - add PROBE into the mix
    - compare travel costs to domain shits.

## References
